In [2]:
import pandas as pd
import numpy as np
import gzip
import os
import glob

## Data cleaning

### Looping over every zipped state data and read as csv

In [14]:
# Don't run this part!!! The raw data is on Shining's laptop locally

zipfiles = "/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_shipment/*.gz"

filelist = glob.glob(zipfiles)

select_data = []

for gzfile in filelist:
    raw = pd.read_csv(gzfile, iterator= True, chunksize = 10000, compression='gzip', error_bad_lines=False)


    for df in raw:

        df["TRANSACTION_DATE1"] = pd.to_datetime(df["TRANSACTION_DATE"], format='%m%d%Y') # Convert transaction data to standardized date time

        df["year"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).year # Extract year

        df = df[["BUYER_COUNTY", "BUYER_STATE", "year", "MME"]] # Subset the columns that we need

        select_data.append(df) # Append the chunks to the list



whole_data = pd.concat(select_data) # Concatenate data

grouped = whole_data.groupby(["BUYER_STATE", "BUYER_COUNTY", "year"], as_index=False).sum("MME")



# Test if the data is correctly loaded

print(len(grouped))

grouped["BUYER_STATE"].value_counts()

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


8736


TX    2037
GA    1366
MS     729
OK     693
AL     603
FL     600
LA     571
MT     430
SC     414
ID     364
WA     351
OR     312
NM     266
Name: BUYER_STATE, dtype: int64

## Read parquet data (Run this part so that you don't need to go over the large loop above)

In [6]:
grouped = pd.read_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/opioid_whole_final.parquet", engine='fastparquet')

In [11]:
len(grouped)

8736

In [8]:
# Test if each row represent a unique unit of data
assert len(grouped[grouped.duplicated(["BUYER_STATE", "BUYER_COUNTY", "year"], keep=False)==True]) == 0

In [10]:
# Load a random row to see if the dataset looks correct
grouped[(grouped["year"]==2006)& (grouped["BUYER_COUNTY"]=="BERNALILLO")]

,BUYER_STATE,BUYER_COUNTY,year,MME
4663,NM,BERNALILLO,2006,1.915714e+08


### Store data in parquet form

In [16]:
grouped.to_parquet('/Users/yangshining/pds2021-opioids-pds6/10_modified_data/opioid_whole_final.parquet', engine='fastparquet')